# BMTC

> Bengaluru Metropolitan Transport Corporation (BMTC) is a state-owned public road transport corporation in the Indian city of Bengaluru.

In [ ]:
#| default_exp bmtc

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import string
import json
import time
from pathlib import Path
import datetime

import logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

import requests
import pandas as pd

In [ ]:
#| hide

# Inside the data directory, there is a directory to store BMTC data.
data_directory = Path('../data/bmtc/')
data_directory.mkdir(exist_ok=True, parents=True)

# Routes

In [ ]:
#| export
def fetch_routes(pattern: str = ""):
    logging.info(f"Fetching routes for pattern = '{pattern}'")
    url = "https://bmtcmobileapi.karnataka.gov.in/WebAPI/SearchRoute_v2"

    headers = {
        "Content-Type": "application/json",
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36",
        "lan": "en"
    }

    if pattern != '':
        payload = {"routetext": pattern}
        try:
            response = requests.post(url, headers=headers, json=payload)
            response.raise_for_status()
            return response.json()['data']
        except requests.exceptions.RequestException as e:
            print(f"Error: {e}")
            return None
    else:
        routes = []
        characters = string.digits + string.ascii_lowercase
        for pattern in characters:
            time.sleep(0.1)
            routes += fetch_routes(pattern)
        return routes

In [ ]:
#| hide
routes = fetch_routes()

# De-duplicate to get unique routes.
routes_df = pd.DataFrame(routes)
routes_df = routes_df.drop_duplicates(subset=["routeno"], keep="first")

routes_df.to_csv(data_directory / 'raw' / 'routes.csv', index=False)

print(routes_df.shape)
routes_df.head(3)

In [ ]:
#| hide
routes_df = pd.read_csv(data_directory / 'raw' / 'routes.csv')

routes_df = routes_df \
    .rename(columns={'routeparentid': 'route_parent_id', 'routeno': 'route_number'})[['route_number', 'route_parent_id']] \
    .sort_values(by = 'route_number') \
    .reset_index(drop = True)

routes_df.to_csv(data_directory / 'cleaned' / 'routes.csv', index=False)

print(routes_df.shape)
routes_df.head(3)

# Route Points

In [ ]:
#| export
def fetch_route_points(route_id: str):
    time.sleep(0.1)
    logging.info(f"Fetching route points for route ID = '{route_id}'")
    url = "https://bmtcmobileapi.karnataka.gov.in/WebAPI/RoutePoints"

    headers = {
        "Content-Type": "application/json",
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36",
        "lan": "en"
    }

    payload = {"routeid": route_id}
    try:
        response = requests.post(url, headers=headers, json=payload)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        return None

# Vehicles

In [ ]:
#| export
def fetch_vehicles(pattern: str = ""):
    time.sleep(0.1)
    logging.info(f"Fetching vehicles for pattern = '{pattern}'")
    url = "https://bmtcmobileapi.karnataka.gov.in/WebAPI/ListVehicles"

    headers = {
        "Content-Type": "application/json",
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36",
        "lan": "en"
    }

    if pattern != '':
        payload = {"vehicleRegNo": pattern}
        try:
            response = requests.post(url, headers=headers, json=payload)
            response.raise_for_status()
            return response.json()['data']
        except requests.exceptions.RequestException as e:
            print(f"Error: {e}")
            return None
    else:
        vehicles = []
        characters = string.digits + string.ascii_lowercase
        for pattern in characters:
            time.sleep(0.1)
            vehicles += fetch_vehicles(pattern)
        return vehicles

In [ ]:
#| hide
vehicles = fetch_vehicles()

# De-duplicate to get unique vehicles.
vehicles_df = pd.DataFrame(vehicles)
vehicles_df = vehicles_df.drop_duplicates(subset=["vehicleregno"], keep="first")

vehicles_df.to_csv(data_directory / 'raw' / 'vehicles.csv', index=False)
print(vehicles_df.shape)
vehicles_df.head(3)

In [ ]:
#| hide
vehicles_df = pd.read_csv(data_directory / 'raw/vehicles.csv')

vehicles_df = vehicles_df \
    .rename(columns={'vehicleid': 'vehicle_id', 'vehicleregno': 'registration_number'})[['vehicle_id', 'registration_number']] \
    .sort_values(by = 'vehicle_id') \
    .reset_index(drop = True)

vehicles_df.to_csv(data_directory / 'cleaned' / 'vehicles.csv', index=False)

print(vehicles_df.shape)
vehicles_df.head(3)

# Trip details

In [ ]:
# | export
def fetch_trip_details(vehicle_id: int):
    time.sleep(0.1)
    logging.info(f"Fetching trip details for vehicle ID = '{vehicle_id}'")
    url = "https://bmtcmobileapi.karnataka.gov.in/WebAPI/VehicleTripDetails_v2"

    headers = {
        "Content-Type": "application/json",
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36"
    }

    payload = json.dumps({"vehicleId": int(vehicle_id)})
    try:
        response = requests.post(url, headers=headers, data=payload)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        print("Response text:", getattr(e.response, "text", None))
        return None

In [ ]:
#| hide
directory = data_directory / 'trip_details' / str(int(datetime.datetime.now().timestamp())) / 'raw'
directory.mkdir(exist_ok=True, parents=True)

In [ ]:
#| hide
for index, row in vehicles_df.iterrows():
    trip_details = fetch_trip_details(vehicle_id = row['vehicle_id']) 
    with open(directory / f"{row['vehicle_id']}.json", "w") as f:
        json.dump(trip_details, f, indent = 4)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()